# Decorators
Decorators provide a simple syntax for extending the behavior of one function to another. More specifically, a decorator is any callable object<sup>1</sup> that modifies a function or a class. A reference to the function or class that you want to modify is passed to the decorator and the decorator returns a modified version of that function or class. These modified functions or classes usually contain calls to the original function or class. Thus we have two different kinds of decorators:
 - Function decorators
 - Class decorators
We'll cover both of these, as well as some prerequisite topics to make sure we're ready to understand perhaps Python's most elegant yet complex concepts.

<sub>1. A callable object is definitionally any object with the special `__call__()` method. Callable objects accept arguments and (possibly) return an object or a tuple of objects. A function is the simplest callable object in Python, but there are others, such as classes or certain class instances.</sub>

## Chaining Decorators in Python
Multiple decorators can be chained in Python.

This is to say, a function can be decorated multiple times with different (or same) decorators. We simply place the decorators above the desired function.

In [1]:
def star(func):
    def inner(*args, **kwargs):
        print("*" * 30)
        func(*args, **kwargs)
        print("*" * 30)
    return inner

def percent(func):
    def inner(*args, **kwargs):
        print("%" * 30)
        func(*args, **kwargs)
        print("%" * 30)
    return inner

@star
@percent
def printer(msg):
    print(msg)
printer("Hello")

******************************
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
Hello
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
******************************


Lots of interesting decorator examples can be found [here](https://wiki.python.org/moin/PythonDecoratorLibrary).